# Installation

Use this: https://riverml.xyz/latest/api/tree/iSOUPTreeRegressor/

In [ ]:
# Install a pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install river --user

In [1]:
import numpy
import pandas as pd
import json
from datetime import datetime

In [2]:
# load static data
def load_static():
    static_df = pd.read_csv(".\\data\\static\\staticFeatures.csv", sep=';', encoding='utf-8', index_col=None)
    static_df.drop('Unnamed: 10', axis='columns', inplace=True)
    static_df['timestamp'] = pd.to_datetime(static_df['timestamp'], infer_datetime_format=True)
    static_df.set_index('timestamp', inplace=True)
    return static_df

static_df = load_static()

In [3]:
# flatten json
def flatten_json(y):
    out = {}
    
    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], a + name)
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + "_")
                i += 1
        else:
            out[name[:-1]] = x
            
    flatten(y)
    return out

# load weather data
def load_weather():
    with open(".\\data\\weather\\converted.json") as f:
        lines = f.read().splitlines()

    df_inter = pd.DataFrame(lines)
    df_inter.columns = ['json_element']
    df_inter['json_element'].apply(json.loads)
    df_final = pd.json_normalize(df_inter['json_element'].apply(json.loads))
    for j in range(df_final.shape[0]):
        print("\r" + str(j), end="")
        for i in range(49):
            if i != 0:
                df_final['hourly.data'][j][i].pop("time", None)
            df_final['hourly.data'][j][i].pop("icon", None)
            df_final['hourly.data'][j][i].pop("apparentTemperature", None)
            df_final['hourly.data'][j][i].pop("uvIndex", None)
            df_final['hourly.data'][j][i].pop("precipType", None)
            df_final['hourly.data'][j][i].pop("summary", None)
            df_final['hourly.data'][j][i].pop("precipIntensity", None)
            df_final['hourly.data'][j][i].pop("precipProbability", None)
            """
            df_final['hourly.data'][0][i]["temperature" + str(i)] = df_final['hourly.data'][0][i].pop("temperature")
            df_final['hourly.data'][0][i]["dewPoint" + str(i)] = df_final['hourly.data'][0][i].pop("dewPoint")
            df_final['hourly.data'][0][i]["humidity" + str(i)] = df_final['hourly.data'][0][i].pop("humidity")
            df_final['hourly.data'][0][i]["pressure" + str(i)] = df_final['hourly.data'][0][i].pop("pressure")
            df_final['hourly.data'][0][i]["windSpeed" + str(i)] = df_final['hourly.data'][0][i].pop("windSpeed")
            df_final['hourly.data'][0][i]["windBearing" + str(i)] = df_final['hourly.data'][0][i].pop("windBearing")
            df_final['hourly.data'][0][i]["cloudCover" + str(i)] = df_final['hourly.data'][0][i].pop("cloudCover", 0.0)
            df_final['hourly.data'][0][i]["visibility" + str(i)] = df_final['hourly.data'][0][i].pop("visibility")
            """

        df_final['hourly.data'][j] = flatten_json(df_final['hourly.data'][j])

    weather_df = pd.json_normalize(df_final['hourly.data'])

    # remove ozone and windGust
    columns = list(weather_df.columns)
    for a in columns:
        if a.find("ozone") != -1:
            weather_df.pop(a)
        if a.find("windGust") != -1:
            weather_df.pop(a)

    # fill in missing values
    weather_df.interpolate(axis=0, limit=None, inplace=True)
    weather_df.fillna(method="backfill", inplace=True)
    weather_df.fillna(method="ffill", inplace=True)

    weather_df['timestamp'] = pd.to_datetime(weather_df['time0']*1000*1000*1000)
    foo = weather_df.pop('time0')
    weather_df.set_index('timestamp', inplace=True)

    # remove duplicates
    weather_df = weather_df.loc[~weather_df.index.duplicated(keep='first')]
    
    return weather_df

weather_df = load_weather()

68

C:\Users\klemen\AppData\Local\Temp/ipykernel_43496/2005317145.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['hourly.data'][j] = flatten_json(df_final['hourly.data'][j])


32712

In [109]:
def load_sensor(n):
    sensor_df = pd.read_json('./data/smartmeters/N' + str(n) + '.json')
    sensor_df.pop("_id")
    sensor_df.pop("node_id")
    sensor_df.pop("stamp_db")
    sensor_df.pop("pg")
    sensor_df.pop("qc")
    sensor_df.pop("qg")
    sensor_df['timestamp'] = pd.to_datetime(sensor_df['stamp'] * 1000 * 1000 * 1000) # unix ts in nanoseconds
    foo = sensor_df.pop("stamp")
    sensor_df.set_index('timestamp', inplace=True)
    
    # enrich sensor_df
    # add 15, 30, 45-minute values
    sensor_df["pc15"] = sensor_df.shift(periods=-1, freq='0.25H')["pc"]
    sensor_df["pc30"] = sensor_df.shift(periods=-2, freq='0.25H')["pc"]
    sensor_df["pc45"] = sensor_df.shift(periods=-3, freq='0.25H')["pc"]
    sensor_df = sensor_df[:-3]

    # add historic values
    moving_averages = ['1H', '6H', '1D', '7D', '30D']
    delays = ['1H', '2H', '3H', '6H', '12H', '1D', '2D', '3D', '7D']

    # create moving averages
    for a in moving_averages:    
        sensor_df['pc_ma_' + a] = sensor_df['pc'].rolling(window=a, min_periods=1).mean()
        sensor_df['pc_std_' + a] = sensor_df['pc'].rolling(window=a, min_periods=1).std()

        # historic values
    for a in moving_averages: 
        for d in delays:
            sensor_df["pc_ma_{}_{}".format(a, d)] = sensor_df.shift(periods=1, freq=d)["pc_ma_{}".format(a)]
    
    return sensor_df

In [117]:
def make_final(n):
    sensor_df = load_sensor(n)
    final_df = pd.concat([sensor_df, static_df], axis=1, join="inner")
    final2_df = pd.concat([weather_df, final_df], axis=1, join="inner")    
    return final_df.dropna()

In [118]:
make_final(1)

,v1,v2,v3,i1,i2,i3,pc,pc15,pc30,pc45,...,pc_ma_30D_7D,timeOfDay,dayAfterHoliday,dayBeforeHoliday,dayOfYear,dayOfWeek,dayOfMonth,holiday,monthOfYear,weekEnd
timestamp,,,,,,,,,,,,,,,,,,,,,
2016-04-13 07:00:00,235.34,234.99,234.02,0.87,1.15,1.09,3975.0,3925.0,4250.0,4850.0,...,4950.000000,7,0,0,104,2,13,0,4,0
2016-04-13 08:00:00,235.13,234.45,234.28,0.96,1.37,1.16,5125.0,5250.0,4975.0,4725.0,...,4825.000000,8,0,0,104,2,13,0,4,0
2016-04-13 09:00:00,233.47,232.70,231.85,0.95,1.28,1.23,5075.0,4425.0,4600.0,5025.0,...,5111.111111,9,0,0,104,2,13,0,4,0
2016-04-13 10:00:00,231.99,231.29,230.51,0.88,1.24,1.18,4875.0,4650.0,4500.0,4475.0,...,5201.923077,10,0,0,104,2,13,0,4,0
2016-04-13 11:00:00,234.49,234.37,234.06,0.36,0.52,0.44,2600.0,2075.0,1500.0,1125.0,...,5016.176471,11,0,0,104,2,13,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-04-04 17:00:00,236.32,236.31,236.49,0.00,0.23,0.04,625.0,575.0,800.0,675.0,...,1520.008681,17,0,0,94,1,4,0,4,0
2017-04-04 18:00:00,236.17,235.99,236.10,0.00,0.28,0.03,750.0,700.0,800.0,1075.0,...,1520.442708,18,0,0,94,1,4,0,4,0
2017-04-04 19:00:00,234.93,234.65,234.99,0.00,0.33,0.03,850.0,600.0,525.0,450.0,...,1520.894097,19,0,0,94,1,4,0,4,0


In [4]:
weather_df

,temperature0,dewPoint0,humidity0,pressure0,windSpeed0,windBearing0,cloudCover0,visibility0,temperature1,dewPoint1,...,temperature48,dewPoint48,humidity48,pressure48,windSpeed48,windBearing48,cloudCover48,visibility48,cloudCover6,cloudCover5
timestamp,,,,,,,,,,,,,,,,,,,,,
2013-12-31 23:00:00,41.68,36.19,0.81,1020.10,5.82,92.0,0.18,6.67,39.92,34.75,...,45.45,43.01,0.91,1017.40,2.000,254.0,1.00,2.92,0.77,0.77
2014-01-01 00:00:00,39.92,34.75,0.82,1019.69,2.83,58.0,0.42,6.18,39.80,34.18,...,42.35,40.77,0.94,1017.74,1.070,116.0,1.00,3.61,0.77,0.77
2014-01-01 01:00:00,39.80,34.18,0.80,1019.30,4.16,112.0,0.18,7.03,39.18,34.93,...,45.81,42.50,0.88,1017.90,0.000,115.0,1.00,2.92,0.75,0.77
2014-01-01 02:00:00,39.18,34.93,0.85,1019.30,9.00,120.0,0.18,7.03,36.02,31.67,...,45.55,43.34,0.92,1018.00,0.815,114.0,1.00,3.09,0.75,0.75
2014-01-01 03:00:00,36.02,31.67,0.84,1019.26,3.25,239.0,0.59,5.68,39.28,34.94,...,43.34,42.25,0.96,1018.21,1.630,113.0,1.00,3.34,0.81,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-24 19:00:00,57.86,55.64,0.92,1020.62,3.69,48.0,0.83,6.22,57.45,55.34,...,60.58,53.38,0.77,1023.39,5.660,56.0,0.20,6.22,1.00,1.00
2017-09-24 20:00:00,57.45,55.34,0.93,1020.91,4.18,48.0,0.83,6.22,57.39,55.14,...,58.86,53.01,0.81,1023.73,5.450,50.0,0.10,6.22,0.92,1.00
2017-09-24 21:00:00,57.39,55.14,0.92,1021.09,4.44,48.0,0.59,6.22,56.97,54.44,...,57.97,52.57,0.82,1023.97,5.220,46.0,0.03,6.22,0.92,0.92
